# Stocker les données avec Elastic Search

## 4. Questions théoriques

### 4.1 Qu’est ce que le sharding, comment pourrait-on imaginer un sharding sur cet index?

Le sharding est une technique utilisée dans les systèmes de bases de données distribuées pour **diviser les données en plusieurs fragments appelés shards**. Chaque shard contient une partie des données et est hébergé sur un nœud du cluster. Le sharding permet de **répartir la charge de travail et de stockage de manière équilibrée entre les nœuds**, ce qui permet d'**améliorer les performances et la capacité de traitement** du système.  
Lorsque vous avez plusieurs “noeuds” dans un cluster Elastic, chaque Shard est répliqué entre les noeuds pour permettre une résilience lorsqu’un des noeuds est défaillant.

Il existe plusieurs approches pour effectuer le sharding :
- basé sur une clé de hachage (aléatoire)
- basé sur une valeur de champ, ici on peut imaginer sharder en fonction du nom/prénom des patients 
- basé sur un critère personnalisé, par exemple dans notre cas on pourrait sharder en fonction de la date des documents


### 4.2 Quelles ingestion de pipelines seraient pertinentes pour notre sujet.

L'ingestion de pipeline fait référence à un ensemble de processus et d'étapes utilisés pour collecter, transformer et charger des données dans un système ou une plateforme. Il s'agit d'une méthodologie structurée qui permet de gérer le flux de données depuis leur source d'origine jusqu'à leur destination finale.

Pipeline de prétraitement du texte :
- Nettoyage du texte : Supprimer les caractères spéciaux, la ponctuation, les balises HTML, etc.
- Normalisation du texte : Convertir le texte en minuscules, supprimer les accents, etc.
- Tokenisation 
- Suppression des stopwords 
- Lemmatisation ou racinisation : Réduire les mots à leur forme de base (lemme) ou à leur racine pour normaliser le texte.

Pipeline d'analyse des sentiments :

- Classification des sentiments : Utiliser des techniques de machine learning pour classer les textes en fonction de leurs sentiments (par exemple, "heureux", "triste", "colère", etc.).
- Estimation de la confiance : Calculer la confiance ou la probabilité associée à la prédiction des sentiments.

Pipeline d'extraction d'entités :

- Reconnaissance des entités nommées : Identifier et extraire des informations spécifiques telles que les noms des patients, les dates, les lieux, etc.
- Détection des entités spécifiques aux émotions : Extraire des entités liées aux émotions, telles que les mots clés ou les expressions spécifiques associées à chaque émotion.

Pipeline d'enrichissement des données :

- Analyse de la similarité des textes : Mesurer la similarité entre les textes pour trouver des documents similaires ou des cas de patients similaires.
- Extraction de concepts : Identifier et extraire des concepts ou des thématiques spécifiques à partir des textes.

## 5. Alternatives

### 5.1 Comment aurait-on pu intégrer la gestion des stopwords au niveau du mapping ? 

Pour intégrer la gestion des stopwords au niveau du mapping, on peut utiliser l'analyseur de texte personnalisé dans le mapping Elasticsearch. Le code ci-dessous nous définissons un nouvel analyseur appelé "custom_analyzer" qui utilise le tokenizer standard et applique 3 filtres : "lowercase", "stop_filter" et "punctuation_filter".  
Le champ "text" est mis à jour avec l'analyseur "custom_analyzer" pour appliquer le filtre de stopwords lors de l'indexation et de la recherche de données.

```
{
  "settings": {
    "analysis": {
      "analyzer": {
        "custom_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "stop_filter",
            "punctuation_filter"
          ]
        }
      },
      "filter": {
        "stop_filter": {
          "type": "stop",
          "stopwords": "_english_"  
        },
        "punctuation_filter": {
          "type": "word_delimiter",
          "generate_word_parts": false,
          "generate_number_parts": false,
          "catenate_words": true,
          "catenate_numbers": true,
          "catenate_all": true,
          "split_on_case_change": true,
          "preserve_original": true,
          "split_on_numerics": false,
          "stem_english_possessive": false,
          "protected_words": []
        }
      }
    }
  },
  "mappings": { notre mapping }
}
```

### 5.2 Comment aurait-on pu intégrer le modèle de ML comme ingest pipeline? (proposez un code)

## 6. Kibana

Pour télécharger Kibana à partir de Docker dans un conteneur et le connecter à Elastic Search, executez le fichier **bash_commands/set_up_kibana.sh**.

Alternative : 

Pour télécharger Kibana à partir du site web et le connecter à notre base de données Elastic Search, suivez les étapes suivantes :
- téléchargez Kibana : https://www.elastic.co/fr/downloads/past-releases/kibana-7-17-10  

- extrayez de l'archive, puis dans le dossier *config*, ouvrir le fichier *kibana.yml*  
- decommentez les lignes `server.host:"localhost"` (7) et `elasticsearch.hosts:["http://localhost:9200"]` (32)
- depuis le dossier d'installation, dans un terminal de commande, entrez la commande `bin/kibana`
- on peut maintenant accéder à kibana à l'adresse : http://localhost:5601.
- allez dans dev tools et lancer la requête :  ` GET notes/_search { "query": { "match_all": {} } }` 
- allez ensuite dans Discover, créer un index pattern, nommé "notes".  
- on a maintenant accès aux données indexées avec Elastic Search.     
- on peut effectuer des requêtes depuis l'onglet **Discover** et créer des visualisation depuis l'onglet **Dashboard**.